In [1]:
import random
import logging
import pandas as pd
import numpy as np
from sklearn.metrics import accuracy_score
from keras.models import Sequential
from keras.layers import Dense
from sklearn.model_selection import train_test_split
from keras import optimizers

logging.basicConfig(level=logging.DEBUG,
                    format='%(asctime)s %(message)s',
                    handlers=[logging.FileHandler("ann_test.log"),
                              logging.StreamHandler()])

In [2]:
class ANN(Sequential):
    
    def __init__(self, child_weights=None):
        super().__init__()

        if child_weights is None:
            layer1 = Dense(8, input_shape=(11,), activation='sigmoid')
            layer2 = Dense(1, activation='sigmoid')
            self.add(layer1)
            self.add(layer2)
        else:
            self.add(
                Dense(
                    8,
                    input_shape=(11,),
                    activation='sigmoid',
                    weights=[child_weights[0], np.ones(8)])
                )
            self.add(
                Dense(
                    1,
                    activation='sigmoid',
                    weights=[child_weights[1], np.zeros(1)])
            )

    def forward_propagation(self, train_feature, train_label):
        predict_label = self.predict(train_feature.values)
        self.fitness = accuracy_score(train_label, predict_label.round())


In [3]:
def crossover(nn1, nn2):
    
    nn1_weights = []
    nn2_weights = []
    child_weights = []

    for layer in nn1.layers:
        nn1_weights.append(layer.get_weights()[0])

    for layer in nn2.layers:
        nn2_weights.append(layer.get_weights()[0])

    for i in range(len(nn1_weights)):
        # Get single point to split the matrix in parents based on # of cols
        split = random.randint(0, np.shape(nn1_weights[i])[1]-1)
        # Iterate through after a single point and set the remaing cols to nn_2
        for j in range(split, np.shape(nn1_weights[i])[1]-1):
            nn1_weights[i][:, j] = nn2_weights[i][:, j]

        child_weights.append(nn1_weights[i])

    mutation(child_weights)
    
    child = ANN(child_weights)
    print("nn1",nn1_weights[0])
    print("nn2",nn2_weights[0])
    return child

In [4]:
def mutation(child_weights):
    selection = random.randint(0, len(child_weights)-1)
    mut = random.uniform(0, 1)
    if mut <= .05:
        child_weights[selection] *= random.randint(2, 5)
    else:
        pass

In [5]:
df=pd.read_csv(r"C:\Users\krish\Downloads\loan.csv")
y=df["Personal Loan"]
x=df.drop(["Personal Loan","Unnamed: 0"],axis=1)
train_feature,test_feature,train_label,test_label=train_test_split(x, y, test_size=0.25, random_state=40)

In [6]:
# store all active ANNs
networks = []
pool = []
# Generation counter
generation = 0
# Initial Population
population = 10
for i in range(population):
    networks.append(ANN())
# Track Max Fitness
max_fitness = 0
# Store Max Fitness Weights
optimal_weights = []

epochs = 10
# Evolution Loop
print(networks)

[<__main__.ANN object at 0x000002266BB5B490>, <__main__.ANN object at 0x000002266BBD76A0>, <__main__.ANN object at 0x000002266CCA8E50>, <__main__.ANN object at 0x000002266CCAA230>, <__main__.ANN object at 0x000002266CCABD60>, <__main__.ANN object at 0x000002266CCCB280>, <__main__.ANN object at 0x000002266CCCBFD0>, <__main__.ANN object at 0x000002266CCE7670>, <__main__.ANN object at 0x000002266CCE79A0>, <__main__.ANN object at 0x000002266CD0F730>]


In [7]:
for i in range(epochs):
    generation += 1
    logging.debug("Generation: " + str(generation) + "\r\n")

    for ann in networks:
        # Propagate to calculate fitness score
        ann.forward_propagation(train_feature, train_label)
        # Add to pool after calculating fitness
        pool.append(ann)

    # Clear for propagation of next children
    networks.clear()

    # Sort anns by fitness
    pool = sorted(pool, key=lambda x: x.fitness)
    pool.reverse()

    # Find Max Fitness and Log Associated Weights
    for i in range(len(pool)):
        if pool[i].fitness > max_fitness:
            max_fitness = pool[i].fitness

            logging.debug("Max Fitness: " + str(max_fitness) + "\r\n")

            # Iterate through layers, get weights, and append to optimal
            optimal_weights = []
            for layer in pool[i].layers:
                optimal_weights.append(layer.get_weights()[0])
            logging.debug('optimal_weights: ' + str(optimal_weights)+"\r\n")

    # Crossover: top 5 randomly select 2 partners
    for i in range(5):
        for j in range(2):
            # Create a child and add to networks
            temp = crossover(pool[i], random.choice(pool))
            # Add to networks to calculate fitness score next iteration
            networks.append(temp)


2023-08-11 20:27:59,180 Generation: 1



118/118 [==============================] - 0s 1ms/step


2023-08-11 20:28:01,910 Max Fitness: 0.9032

2023-08-11 20:28:01,918 optimal_weights: [array([[ 0.54279655, -0.1360439 , -0.18781364, -0.04804862,  0.08149689,
         0.5023722 , -0.05045849,  0.10541469],
       [ 0.0742175 ,  0.45956832,  0.53419393, -0.14583218, -0.21255085,
         0.19253868, -0.00325078,  0.43137634],
       [-0.20075929,  0.5513945 , -0.07525864,  0.31601113, -0.48507553,
         0.49083513, -0.2904123 ,  0.5071575 ],
       [-0.09194648, -0.22960886,  0.22246331, -0.47099307, -0.10204697,
         0.20935875,  0.5308724 ,  0.45683903],
       [-0.36395937,  0.15919328, -0.2528885 ,  0.50856584, -0.13743943,
        -0.5393239 , -0.2177625 , -0.17383179],
       [ 0.01665437, -0.48331425, -0.25248736,  0.19938934, -0.51534593,
        -0.10416183,  0.16817045,  0.08168513],
       [ 0.39061117, -0.446135  ,  0.01143956,  0.01117259, -0.20877811,
        -0.43392664, -0.34719372,  0.08297497],
       [-0.34692252,  0.48088044, -0.14615896,  0.12286282,  0.147

nn1 [[ 0.54279655 -0.1360439  -0.18781364 -0.04804862  0.08149689  0.5023722
   0.23298931  0.10541469]
 [ 0.0742175   0.45956832  0.53419393 -0.14583218 -0.21255085  0.19253868
   0.45485896  0.43137634]
 [-0.20075929  0.5513945  -0.07525864  0.31601113 -0.48507553  0.49083513
  -0.17428452  0.5071575 ]
 [-0.09194648 -0.22960886  0.22246331 -0.47099307 -0.10204697  0.20935875
  -0.09955922  0.45683903]
 [-0.36395937  0.15919328 -0.2528885   0.50856584 -0.13743943 -0.5393239
  -0.34383428 -0.17383179]
 [ 0.01665437 -0.48331425 -0.25248736  0.19938934 -0.51534593 -0.10416183
   0.41169274  0.08168513]
 [ 0.39061117 -0.446135    0.01143956  0.01117259 -0.20877811 -0.43392664
  -0.3905932   0.08297497]
 [-0.34692252  0.48088044 -0.14615896  0.12286282  0.14789665 -0.27867544
  -0.04885703 -0.13207233]
 [ 0.15619433 -0.07007524  0.04757202  0.17145538 -0.2484661  -0.2219361
   0.13228565  0.11634552]
 [-0.23761281  0.4879554   0.38236833  0.4032243   0.4411854  -0.05164355
   0.0933637  -0

2023-08-11 20:28:02,155 Generation: 2



...

nn1 [[ 0.30428147  0.24316919  0.3228007   0.13748616 -0.02929682  0.513702
   0.07672304  0.12811428]
 [ 0.23038834 -0.35269195  0.43781567  0.412553    0.01056844 -0.46903777
  -0.5576963  -0.29188234]
 [-0.2676686   0.29381782 -0.21687329 -0.45478675 -0.1723296  -0.40268767
  -0.5613764   0.26003993]
 [ 0.5506578   0.33753455 -0.35378242  0.38671702 -0.08286601 -0.5433173
   0.1333304  -0.5565715 ]
 [ 0.01810479  0.48748273 -0.37189755 -0.24492422  0.07157272 -0.55529255
   0.05146599  0.32202536]
 [ 0.35310674  0.09046268 -0.54517317 -0.08421385 -0.34226486  0.29942757
  -0.2143443   0.36740923]
 [-0.2815773  -0.31630963 -0.1439274  -0.20252192 -0.03177488  0.4164636
   0.33202678  0.55110663]
 [ 0.37742835 -0.2950125   0.39805198 -0.5052525  -0.12120298  0.08339858
   0.08008128  0.2838891 ]
 [ 0.09528261  0.18519849 -0.2006481  -0.23837757 -0.21451056 -0.22693235
  -0.53467804  0.3203103 ]
 [ 0.08518374 -0.50774205  0.06396604 -0.5552132  -0.23622853  0.49524826
   0.16557598 -0.

2023-08-11 20:28:22,154 Generation: 9



118/118 [==============================] - 0s 1ms/step
nn1 [[ 0.30428147  0.24316919  0.3228007   0.13748616  0.08149689  0.5023722
   0.07672304  0.12811428]
 [ 0.23038834 -0.35269195  0.43781567  0.412553   -0.21255085  0.19253868
  -0.5576963  -0.29188234]
 [-0.2676686   0.29381782 -0.21687329 -0.45478675 -0.48507553  0.49083513
  -0.5613764   0.26003993]
 [ 0.5506578   0.33753455 -0.35378242  0.38671702 -0.10204697  0.20935875
   0.1333304  -0.5565715 ]
 [ 0.01810479  0.48748273 -0.37189755 -0.24492422 -0.13743943 -0.5393239
   0.05146599  0.32202536]
 [ 0.35310674  0.09046268 -0.54517317 -0.08421385 -0.51534593 -0.10416183
  -0.2143443   0.36740923]
 [-0.2815773  -0.31630963 -0.1439274  -0.20252192 -0.20877811 -0.43392664
   0.33202678  0.55110663]
 [ 0.37742835 -0.2950125   0.39805198 -0.5052525   0.14789665 -0.27867544
   0.08008128  0.2838891 ]
 [ 0.09528261  0.18519849 -0.2006481  -0.23837757 -0.2484661  -0.2219361
  -0.53467804  0.3203103 ]
 [ 0.08518374 -0.50774205  0.063966

2023-08-11 20:28:24,969 Generation: 10



nn1 [[ 0.30428147  0.24316919 -0.4496922   0.14150703  0.01597947  0.5023722
   0.23298931  0.12811428]
 [ 0.23038834 -0.35269195 -0.17185706  0.02468336  0.4216789   0.19253868
   0.45485896 -0.29188234]
 [-0.2676686   0.29381782 -0.4290392   0.07025266  0.22344124  0.49083513
  -0.17428452  0.26003993]
 [ 0.5506578   0.33753455  0.27574235 -0.04718989  0.42937684  0.20935875
  -0.09955922 -0.5565715 ]
 [ 0.01810479  0.48748273  0.01223344  0.5290051  -0.03278375 -0.5393239
  -0.34383428  0.32202536]
 [ 0.35310674  0.09046268  0.3727045   0.22315079  0.3419544  -0.10416183
   0.41169274  0.36740923]
 [-0.2815773  -0.31630963  0.33082914  0.42959696  0.33357763 -0.43392664
  -0.3905932   0.55110663]
 [ 0.37742835 -0.2950125   0.49962932  0.2383411  -0.28685984 -0.27867544
  -0.04885703  0.2838891 ]
 [ 0.09528261  0.18519849 -0.17539963  0.3563503  -0.09779724 -0.2219361
   0.13228565  0.3203103 ]
 [ 0.08518374 -0.50774205  0.11752951  0.22475678  0.48224884 -0.05164355
   0.0933637  -0

In [9]:
ann = ANN(optimal_weights)
print(optimal_weights)
predict_label = ann.predict(test_feature.values)
print('Test Accuracy: %.2f' % accuracy_score(test_label, predict_label.round()))

[array([[ 0.54279655, -0.1360439 , -0.18781364, -0.04804862,  0.08149689,
         0.5023722 , -0.05045849,  0.10541469],
       [ 0.0742175 ,  0.45956832,  0.53419393, -0.14583218, -0.21255085,
         0.19253868, -0.00325078,  0.43137634],
       [-0.20075929,  0.5513945 , -0.07525864,  0.31601113, -0.48507553,
         0.49083513, -0.2904123 ,  0.5071575 ],
       [-0.09194648, -0.22960886,  0.22246331, -0.47099307, -0.10204697,
         0.20935875,  0.5308724 ,  0.45683903],
       [-0.36395937,  0.15919328, -0.2528885 ,  0.50856584, -0.13743943,
        -0.5393239 , -0.2177625 , -0.17383179],
       [ 0.01665437, -0.48331425, -0.25248736,  0.19938934, -0.51534593,
        -0.10416183,  0.16817045,  0.08168513],
       [ 0.39061117, -0.446135  ,  0.01143956,  0.01117259, -0.20877811,
        -0.43392664, -0.34719372,  0.08297497],
       [-0.34692252,  0.48088044, -0.14615896,  0.12286282,  0.14789665,
        -0.27867544, -0.34178054, -0.13207233],
       [ 0.15619433, -0.0700752